[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/ai-agents-lab-notebooks/blob/main/notebook_template.ipynb)


[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/ai-agents-lab/)


# Step 1: Install libraries


In [ ]:
!pip install -qU langchain langchainhub langchain-community langchain-fireworks \
langchain-huggingface langchain-mongodb arxiv pymupdf duckduckgo-search datasets pymongo

# Step 2: Setup prerequisites

Replace:

- `<CODE_BLOCK_1>` with your **MongoDB connection string**
- `<CODE_BLOCK_2>` with your **Fireworks API key**
- `<CODE_BLOCK_3>` with your **LangSmith API key**


In [ ]:
import getpass
import os

In [ ]:
# Retain the quotes ("") when pasting the URI
MONGODB_URI = "<CODE_BLOCK_1>"

In [ ]:
# Retain the quotes ("") when pasting the API key
os.environ["FIREWORKS_API_KEY"] = "<CODE_BLOCK_2>"

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# Retain the quotes ("") when pasting the API key
os.environ["LANGCHAIN_API_KEY"] = "<CODE_BLOCK_3>"

# Step 3: Create a knowledge base


### Download the dataset from Hugging Face


In [ ]:
import pandas as pd
from datasets import load_dataset
from pymongo import MongoClient

In [ ]:
data = load_dataset("mongodb-eai/arxiv-embeddings")
dataset_df = pd.DataFrame(data["train"])

In [ ]:
# Preview the dataset -- notice that the dataset already has an `embedding` column, which consists of embeddings of the paper abstracts.
dataset_df.head()

### Ingest data into MongoDB


📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/mongo_client.html


In [ ]:
# Initialize a MongoDB Python client
client = <CODE_BLOCK_4>

In [ ]:
# Name of the database -- Change if needed or leave as is
DB_NAME = "mongodb_agents_lab"
# Name of the collection -- Change if needed or leave as is
COLLECTION_NAME = "knowledge"
# Name of the vector search index -- Change if needed or leave as is
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

📚 https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-collection


In [ ]:
# Connect to the collection defined above using the MongoDB client
collection = <CODE_BLOCK_5>

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html


In [ ]:
# Bulk delete all existing records from the collection defined above -- should be a one-liner
<CODE_BLOCK_6>

In [ ]:
# Ingest data into the collection
records = dataset_df.to_dict("records")

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html


In [ ]:
# Bulk insert `records` into the collection defined above -- should be a one-liner
<CODE_BLOCK_7>

print("Data ingestion into MongoDB completed")

# Step 4: Create a vector search index

Follow the instructions in the documentation to create a Vector Search index in the Atlas UI.


# Step 5: Instantiate chat completion LLM


In [ ]:
from langchain_fireworks import ChatFireworks

📚 https://python.langchain.com/v0.1/docs/integrations/chat/fireworks/


In [ ]:
# Set the temperature for the chat model to 0.0 and max tokens to 1024
llm = <CODE_BLOCK_8>

# Step 6: Create agent tools


In [ ]:
from langchain.tools import tool
from langchain_community.document_loaders import ArxivLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### Tool to fetch paper metadata from Arxiv


In [ ]:
@tool
def get_paper_metadata_from_arxiv(topic: str) -> list:
    """
    Fetch and return paper metadata for 5 Arxiv papers matching the given topic, for example: Retrieval Augmented Generation.

    Args:
    topic (str): The topic to find papers for on Arxiv.

    Returns:
    list: Metadata about the papers matching the topic.
    """
    docs = ArxivLoader(query=topic, load_max_docs=5).load()
    # Extract just the metadata from each document
    metadata = [doc.metadata for doc in docs]
    return metadata

### Tool to fetch the summary of a paper

📚 https://python.langchain.com/v0.1/docs/integrations/document_loaders/arxiv/

📚 https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html


In [ ]:
@tool
def get_paper_summary_from_arxiv(id: str) -> str:
    """
    Fetch and return the summary for a single research paper from Arxiv given the paper ID, for example: 1605.08386.

    Args:
    id (str): The paper ID.

    Returns:
    str: Summary of the paper.
    """
    # Create a tool that uses the `ArxivLoader` document loader to return the paper summary given the paper ID (`id`).
    # NOTE:
        # Use the `get_summaries_as_docs` method of `ArxivLoader`
        # Handle the case where the paper ID is invalid i.e., the number of docs returned from `ArxivLoader` are 0.
    <CODE_BLOCK_9>

### Tool to answer questions based on information in the knowledge base

In **Step 3**, we created a knowledge base for the agent. This tool should use the knowledge base to help the agent answer questions about topics. To do this, you will need to:

- Create a MongoDB vector store

- Get relevant documents from the vector store

- Create a RAG chain that uses the retrieved documents and the LLM from **Step 5** to answer questions


#### Create a MongoDB Vector Store


In [ ]:
# Embedding model to use for the vector store -- DO NOT CHANGE
embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

📚 https://api.python.langchain.com/en/latest/_modules/langchain_mongodb/vectorstores.html#MongoDBAtlasVectorSearch


In [ ]:
# Create a MongoDBAtlas vector store
# Use the `from_connection_string` method of the MongoDBAtlasVectorSearch class.
# Arguments: connection_string, namespace, embedding, index_name, text_key
# NOTE: Use variables defined in Steps 2, 3 and this step as values for the above arguments
vector_store = <CODE_BLOCK_10>

#### Get relevant documents from the vector store


📚 https://github.com/langchain-ai/langchain/blob/master/libs/partners/mongodb/langchain_mongodb/vectorstores.py#L187


In [ ]:
# Define a function to retrieve documents with a similarity score greater than 0.8 from the `vector_store`
# Use the `similarity_search_with_score` method to get similar documents with their scores
# Filter the retrieved list of documents to only return those with a score > 0.8
def get_context(query):
    <CODE_BLOCK_11>

#### Create a RAG chain


📚 https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

📚 https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html


In [ ]:
@tool
def answer_questions_about_topics(query: str) -> list:
    """
    Answer questions about a given topic based on information in the knowledge base.

    Args:
    query (str): User query about a topic.

    Returns:
    str: Information about the topic.
    """
    # Create a RAG chain that uses the `llm` we instantiated in Step 4 and the `get_context` function above
    # NOTE: Use `RunnableLambda` to convert the `get_context` function  into a Runnable
    # Return the response of running `invoke` on the chain with `query` as an argument
    <CODE_BLOCK_12>

In [ ]:
# Create the list of tools
tools = [
    get_paper_metadata_from_arxiv,
    get_paper_summary_from_arxiv,
    answer_questions_about_topics,
]

### Test out the tools


In [ ]:
# Test out the `get_paper_metadata_from_arxiv` tool
get_paper_metadata_from_arxiv.invoke("Retrieval Augmented Generation")

In [ ]:
# Test out the `get_paper_summary_from_arxiv` tool with paper ID 1808.09236
<CODE_BLOCK_13>

In [ ]:
# Test out the `get_paper_summary_from_arxiv` with an invalid paper ID eg: 808.09236
<CODE_BLOCK_14>

In [ ]:
# Test out the `answer_questions_about_topics` tool with the topic "Partial cubes"
<CODE_BLOCK_15>

In [ ]:
# Test out the `answer_questions_about_topics` tool with a topic that is not present in the knowledge base eg:"Tree of Thoughts"
<CODE_BLOCK_16>

# 🦹 Use web search to get information


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.docstore.document import Document

📚 https://python.langchain.com/v0.2/docs/integrations/tools/ddg/

📚 https://stackoverflow.com/questions/76551067/how-to-create-a-langchain-doc-from-an-str


In [ ]:
# Extend the `get_context` function from Step 6 to use DuckDuckGo search if no documents are retrieved from the knowledge base
# NOTE: The result from DuckDuckGo search is a string and will need to be converted to a LangChain document for downstream use
def get_context(query):
    <CODE_BLOCK_17>

In [ ]:
answer_questions_about_topics.invoke("Tree of Thoughts")

# Step 7: Create a basic tool-calling agent


In [ ]:
from langchain.tools.render import render_text_description
from langchain_core.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

📚 https://api.python.langchain.com/en/latest/tools/langchain.tools.render.render_text_description.html


In [ ]:
# Try out a simple system prompt
# Use the `render_text_description` method to include the list of tools the agent can access, in the prompt.
system_message = <CODE_BLOCK_18>

### 🦹 CoT prompting


In [ ]:
# system_message = f"""Given a question, write out in a step-by-step manner your reasoning
# for how you will solve the problem to be sure that your conclusion is correct.
# Avoid simply stating the correct answer at the outset. You can answer directly if the user
# is greeting you or similar. Otherwise, you have access to the following tools:

# {render_text_description(tools)}

# Begin!
# """

📚 https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/


In [ ]:
# Refer to the `Create Agent` and `Run Agent` sections in the docs above to craft a prompt, initialize an agent, and an agent executor
# NOTE:
    # Use the `system_message` above as the system prompt
    # Do not include chat history in the prompt right now
    # Name the agent executor object `agent_executor`
<CODE_BLOCK_19>

📚 https://python.langchain.com/v0.1/docs/modules/agents/how_to/handle_parsing_errors/


In [ ]:
# Test that the agent works as expected.
# If it runs into parsing errors, add appropriate arguments to the `agent_executor` object and try again.
agent_executor.invoke({"input": "Give me papers on the topic prompt compression."})

# Step 8: Create a ReAct agent


In [ ]:
from langchain import hub
from langchain.agents import create_react_agent

In [ ]:
# Pull a ready-made react prompt from LangChain hub -- Modify if needed
prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

📚 https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/


In [ ]:
# Refer to the above docs to initialize a ReAct agent and the agent executor
# NOTE:
    # Use the `prompt` above as the agent prompt
    # Do not include chat history in the prompt right now
    # Name the agent executor object `agent_executor`
<CODE_BLOCK_20>

📚 https://python.langchain.com/v0.1/docs/modules/agents/how_to/handle_parsing_errors/


In [ ]:
# Test that the agent works as expected.
# If it runs into parsing errors, add appropriate arguments to the agent executor and try again.
agent_executor.invoke({"input": "Give me the summary for the paper 1808.09236."})

# 🦹 Create a custom agent without using abstractions


In [ ]:
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import (
    format_to_tool_messages,
)

📚 https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/


In [ ]:
# Refer to the docs above to build a custom agent without using the `create_tool_calling_agent` abstraction.
# NOTE:
    # Do not include chat history in the prompt right now
    # Name the agent executor object `agent_executor`
<CODE_BLOCK_21>

In [ ]:
agent_executor.invoke({"input": "Give me papers on the topic prompt compression."})

# Step 9: Add memory to agents using MongoDB


In [ ]:
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

📚 https://python.langchain.com/v0.2/docs/integrations/memory/mongodb_chat_message_history/


In [ ]:
# Define a function that returns an instance of `MongoDBChatMessageHistory`
# Use variables defined in Steps 2 & 3 for the `connection_string` and `database_name` parameters
# Choose a collection name of your choice to store the chat history
def get_message_history(session_id: str) -> MongoDBChatMessageHistory:
    <CODE_BLOCK_22>

📚 https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/#create-agent


In [ ]:
# Refer to the above docs and modify the code below to include chat history
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

{render_text_description(tools)}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        <CODE_BLOCK_23>,
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

📚 https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/#langsmith


In [ ]:
# Look at the example in the docs above to add and manage chat history for the agent.
agent_with_chat_history = <CODE_BLOCK_24>

In [ ]:
# Test the agent with chat history to make sure it works
agent_with_chat_history.invoke(
    {"input": "Get me papers on Prompt Compression."},
    config={"configurable": {"session_id": "my-session"}},
)

In [ ]:
# Invoke the agent with a follow-up question to make sure that the chat history is being used
<CODE_BLOCK_25>